In [1]:
import numpy as np
import os 
import sys
import matplotlib.pyplot as plt
from matplotlib import colors
from os import listdir
from scipy.signal import hilbert
from scipy import signal
from scipy.interpolate import interp1d, interp2d, interpn
import numpy.matlib
import math



In [2]:
Path =  '/Users/gijsdepooter/Desktop/Desktop - Gijs’s MacBook Pro/UROP/RFArrays/'
F_num = 1.5
f_resample = 20  # MHz
c = 1500 /1000 # mm/s
tstart = 0
terror = 0
ne = 32
pitch_mm = 0.3


In [3]:
def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_highpass_filter(data, cutoff, fs, order=5):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = signal.filtfilt(b, a, data)
    return y

In [4]:
#Grid
x_channels_index = np.arange(0,ne)
x_centre = (ne-1)/2
x_channels_norm = x_channels_index - x_centre

x_channels_mm = x_channels_norm   * pitch_mm

x_axis_mm = np.linspace(-10,10,100)
z_grid = np.arange(0,3800)
z_res = 0.01
z_axis_mm =  z_grid*z_res
x_axis_mm = x_axis_mm + x_centre*pitch_mm
x_channels_mm = x_channels_mm + x_centre*pitch_mm 

In [5]:
files = listdir(Path)
print(len(files))

81


In [6]:

def bf(data,x_axis,z_axis,x_channels,ne,theta,c,terror,tstart,f_resample,mask):


    X = np.matlib.repmat(x_axis, len(z_axis), 1)
    Z = np.matlib.repmat(z_axis, len(x_axis),1).T

  
    # Delay matrix
    delay_tx = np.zeros_like(X)
    
    delay_rcv = np.zeros((X.shape[0],X.shape[1],ne))
    
    t = np.arange( 0 , data.shape[0]/f_resample, 1/f_resample) + tstart-terror
    interp_value = np.zeros((X.shape[0],X.shape[1],ne))

    # Calculate transmit delays **********************************************************************************
    
    delay_tx = ( Z*math.cos(math.radians(theta)) + X*math.sin(math.radians(theta)) )/c
    
    #  Beamforming **************************************************************************************
    
    for j in range(0,ne):
        
        delay_rcv[:,:,j] = np.sqrt((X - x_channels[j])**2 + Z**2 )/c
        Delay = delay_tx + delay_rcv[:,:,j]
        f = interp1d(t, data[:,j],fill_value=0,bounds_error=False,kind='linear')
        intd = f(Delay.flatten()).reshape(X.shape)
        interp_value[:,:,j] = intd

        
    
    return np.sum(interp_value*mask,axis=2)
   
    
    


In [7]:
mask =np.ones((z_axis_mm.shape[0],x_axis_mm.shape[0],ne))

In [10]:
pr_IMG = np.zeros((1024,32,len(files)))
bf_IMG = np.zeros((z_axis_mm.shape[0],x_axis_mm.shape[0],len(files)))

for j in range(0,len(files)):
    fileName = str(files[j]).replace(".npy","")
    file_comp = str(fileName).replace(",", ".").split("_")
    angle = file_comp[2]
    angle = float(angle)
    X = np.load(Path +files[j] )
    X = butter_highpass_filter(X.T,1*1e6,20*1e6,order =5).T   # MUST BE ROW ARRAY 32*1000
    pr_IMG[:,:,j]=X
    bf_IMG[:,:,j]=bf(X,x_axis_mm,z_axis_mm,x_channels_mm,ne,angle,c,0,tstart,20,mask)
    print(len(X))
    
    
    
   
    
    

1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024


KeyboardInterrupt: 

In [9]:
comp_IMG = np.sum(bf_IMG,axis=2)

NameError: name 'bf_IMG' is not defined

In [12]:
z_start = 200
z_end = z_start + 3500
range = 5
x_res = (x_axis_mm[-1]-x_axis_mm[0])/len(x_axis_mm)
x_cen_ind = np.round(len(x_axis_mm)/2).astype(int)
x_range_ind = (range/x_res).astype(int)
x_start = x_cen_ind - x_range_ind
x_end = x_cen_ind + x_range_ind - 1
print(x_res,len(x_axis_mm)/2,x_start,x_end )
dB = 40

# Calculate the enevlope, normalise and log compress the results.
YY =comp_IMG[z_start:z_end,x_start:x_end]
img = hilbert(YY.T).T  
img = np.abs(img)
img= img/np.amax(img)
img_dB = 20*np.log10(img)


plt.imshow(img_dB, cmap='gray',  aspect=1, extent=[x_axis_mm[x_start],x_axis_mm[x_end],z_axis_mm[z_end],z_axis_mm[z_start]],vmin=-dB, vmax=0)
plt.show()

0.2 50.0 25 74


NameError: name 'comp_IMG' is not defined